In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline

In [2]:
# Sample
from transformers.response_var import EnrolledOneYearLater
from transformers.filter_term_table import FilterTermRows
from transformers.add_student_info import AddStudentInfo

In [3]:
# Features
from transformers.features_gender import FeatureIsFemale
from transformers.features_age import FeatureAge
from transformers.features_gpa import FeatureGPA
from transformers.features_ethnicity import FeatureEthnicGroupDummy
from transformers.features_sat_act import FeatureSatAct
from transformers.features_residency import FeatureStateResident
from transformers.features_parent_education import FeatureParentEducation
from transformers.features_warning_signals import FeatureCourseWarningSignals
from transformers.features_acad_load import FeatureAcadLoad
from transformers.features_family_income import FeatureFamilyIncome

# Params

In [4]:
min_year = 2015

# Data Loading

In [5]:
files = ! ls /data0/larc/*.csv

In [6]:
for file in files:
    print(file)

/data0/larc/LARC Student_LARC_20230125_STDNT_INFO.csv
/data0/larc/LARC Student_LARC_20230125_STDNT_TERM_CLASS_INFO.csv
/data0/larc/LARC Student_LARC_20230125_STDNT_TERM_INFO.csv
/data0/larc/LARC Student_LARC_20230125_STDNT_TERM_TRNSFR_INFO.csv


In [7]:
stdn_info = pd.read_csv(files[0], low_memory=False)

In [8]:
term  = pd.read_csv(files[2], low_memory=False)

In [9]:
term_class = pd.read_csv(files[1], low_memory=False)

In [10]:
term_class.head()

,#SNPSHT_RPT_DT,STDNT_ID,TERM_CD,CLASS_NBR,TERM_SHORT_DES,GRD_BASIS_ENRL_CD,GRD_BASIS_ENRL_DES,CRSE_GRD_INPUT_CD,CRSE_GRD_OFFCL_CD,UNITS_TAKEN_NBR,...,CLASS_MTG_TUES_IND,CLASS_MTG_WED_IND,CLASS_MTG_THURS_IND,CLASS_MTG_FRI_IND,CLASS_MTG_SAT_IND,CLASS_MTG_SUN_IND,CLASS_SCTN_TOPIC_DES,CLASS_GPA,CRSE_GPA,CLASS_CSP_IND
0,2023-01-25,2,2110,10411,FA 2016,GRD,Graded,A,A,4.0,...,1,0,1,0,0,0,,3.253,3.253,0
1,2023-01-25,2,2160,25852,FA 2017,OPF,Optional Pass/Fail,B+,P,4.0,...,1,0,1,0,0,0,,2.879,3.016,0
2,2023-01-25,131072,1260,13804,FA 1999,GRD,Graded,A-,A-,3.0,...,1,1,1,0,0,0,,8.250,8.250,0
3,2023-01-25,131072,1270,13407,WN 2000,GRD,Graded,C-,C-,2.0,...,0,1,0,0,0,0,,4.060,4.060,0
4,2023-01-25,139264,2020,21393,WN 2015,GRD,Graded,B-,B-,3.0,...,1,0,1,0,0,0,,3.594,3.495,0


In [11]:
# Filter columns
student_cols = [
    'STDNT_ID', 'STDNT_BIRTH_YR', 'STDNT_BIRTH_MO',
    # Student Sex Code (1 = Female; 2 = Male; 3 = Unknown)
    'STDNT_SEX_CD',
    # A code indicating whether a student has learned and used
    # English from early childhood (1 = Yes; 0 = No)
    'STDNT_NTV_ENG_SPKR_IND',
    # A code representing a student's race as defined by the federal government's requirements
    'STDNT_ETHNC_GRP_CD',
    # A code indicating whether a student has self-identified as Hispanic or Latino using the
    # federal standards. Students who have self-identified as Hispanic or Latino will be
    # categorized as such even if they identify other races
    'STDNT_HSPNC_LATINO_IND',
    # A code indicating whether a student has self-identified as
    # having Asian ethnicity (1 =Yes; 0 = No)
    'STDNT_ASIAN_IND',
    'STDNT_BLACK_IND',
    'STDNT_HWIAN_IND',
    'STDNT_HSPNC_IND',
    'STDNT_NTV_AMRCN_IND',
    'STDNT_WHITE_IND',
    'STDNT_MULTI_ETHNC_IND',
    # Student International Indicator (1 = Yes; 0 = No)
    'STDNT_INTL_IND',
    # Parent Maximum Education Level (Examples of Valid Values:
    # Bachelor's degree; Master's degree; High School diploma)
    'PRNT_MAX_ED_LVL_DES',
    # Parent Dependent Number
    'PRNT_DEP_NBR_CD',
    # Estimated Gross Family Income (Examples of Valid Values:
    # $25,000 - $49,999; $50,000 - $74,999; $75,000 - $99,999)
    'EST_GROSS_FAM_INC_DES', 
    # Single Parent Indicator
    'SNGL_PRNT_IND',
    # SAT
    'MAX_SATI_TOTAL_CALC_SCR', 'MAX_SATI_TOTAL_MSS_ERWS_SCR',
    'MAX_SATI_TOTAL_MSS_ERWS_PCTL',
    'MAX_SATI_MATH_SCR', 'MAX_SATI_MSS_SCR',
    'MAX_SATI_MATH_PCTL', 'MAX_SATI_MSS_PCTL',
    # ACT
    'MAX_ACT_COMP_SCR', 'MAX_ACT_COMP_PCTL',
    'MAX_ACT_MATH_PCTL', 
    # High school GPA
    'HS_GPA',
    # High School Calculus Indicator
    'HS_CALC_IND'
]

In [12]:
term_cols = [
    'STDNT_ID', 
    'TERM_CD', 'TERM_SHORT_DES',
    # The number of academic careers a student is registered for within a term
    'ACAD_CRER_CNT',
    # Primary Career based on enrolment hours (e.g. Undergraduate L S & A; Undergraduate Engineering; Rackham)
    'PRMRY_CRER_CD',
    'PRMRY_CRER_DES',
    # The level of a student's primary career. E.g: U = Undergraduate; G = Graduate; 
    # P = Graduate-Professional; A = Academic Outreach
    'CRER_LVL_CD',
    
    # Examples of valid values: Lit, Science & Arts UG Degree; Biology Mas; Psychology Doc
    'PGM_1_DES',
    'PGM_1_MAJOR_1_CIP_DES',
    'PGM_2_DES',
    'PGM_2_MAJOR_1_CIP_DES',
    'PGM_3_DES',
    'PGM_3_MAJOR_1_CIP_DES',
        
    # Student's residency for fee assessment purposes (InState; Out-State)
    'RES_SHORT_DES',
    # Entry type (readmission can indicate higher risk of dropping out)
    # Categorical (Continuing; Freshman; Readmit/Cr)
    'ENTRY_TYP_SHORT_DES',
    'CURR_GPA',
    'CUM_GPA',
    # A code indicating whether a student is enrolled in any honors
    # program during the term (1 = Yes; 0 = No)
    'HONORS_PGM_IND',
    # Student's academic load based on units taken for the term
    # (Full-Time; Half-Time; No Units)
    'ACAD_LOAD_SHORT_DES',
    # A code indicating whether a student is enrolled in any
    # supplemental study plans
    'SPPLMNT_STUDY_IND'
]

In [13]:
stdn_info_df = stdn_info.copy()[student_cols]
term_df = term.copy()[term_cols]

# Sample

In [14]:
sample_pipeline = Pipeline([
    ('compute_y_variable', EnrolledOneYearLater()),
    ('filter_term_rows', FilterTermRows(min_year)),
    ('add_student_info', AddStudentInfo(stdn_info_df)),
])

# Fit and transform your data using the pipeline
sample_df = sample_pipeline.fit_transform(term_df)

In [15]:
sample_df.head(3)

,TERM_CD,TERM_SHORT_DES,ACAD_CRER_CNT,PRMRY_CRER_CD,PRMRY_CRER_DES,CRER_LVL_CD,PGM_1_DES,PGM_1_MAJOR_1_CIP_DES,PGM_2_DES,PGM_2_MAJOR_1_CIP_DES,...,MAX_SATI_TOTAL_MSS_ERWS_PCTL,MAX_SATI_MATH_SCR,MAX_SATI_MSS_SCR,MAX_SATI_MATH_PCTL,MAX_SATI_MSS_PCTL,MAX_ACT_COMP_SCR,MAX_ACT_COMP_PCTL,MAX_ACT_MATH_PCTL,HS_GPA,HS_CALC_IND
33128,2060,FA 2015,1,ULSA,Undergraduate L S & A,U,"Lit, Sci, and the Arts UG Deg",German Language and Literature,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,33.0,91.0,71.0,3.9,1
33129,2110,FA 2016,1,ULSA,Undergraduate L S & A,U,"Lit, Sci, and the Arts UG Deg","Liberal Arts and Sciences, General Studies and...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,32.0,78.0,97.0,3.8,1
126498,2060,FA 2015,1,ULSA,Undergraduate L S & A,U,"Lit, Sci, and the Arts UG Deg","Liberal Arts and Sciences, General Studies and...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,28.0,33.0,4.0,3.9,0


In [ ]:
sample_df.groupby('PRMRY_CRER_DES')[['PRMRY_CRER_DES']].count()

# Features

## Params

In [17]:
age_bins = pd.IntervalIndex.from_tuples([(0, 17.5), (17.5, 19), (19,100)])
hs_gpa_bins = pd.IntervalIndex.from_tuples([(0, 2.7), (2.7, 3.3), (3.3, 3.6), (3.6, 3.8), (3.8, 3.9), (3.9, 5)])
first_term_gpa_bins = pd.IntervalIndex.from_tuples([(-1, 0), (0, 2), (2, 2.7), (2.7, 3.3), (3.3, 3.7), (3.7, 5)])
SAT_bins = pd.IntervalIndex.from_tuples([(0, 1000), (1000, 1200), (1200, 1300), (1300, 1400), (1400, 1600)])

## Features

In [18]:
features_pipeline = Pipeline([
    ('feature_gender', FeatureIsFemale()),
    ('feature_age', FeatureAge(age_bins)),
    ('feature_gpa', FeatureGPA(hs_gpa_bins, first_term_gpa_bins)),
    ('feature_ethnicity', FeatureEthnicGroupDummy()),
    ('feature_sat_act', FeatureSatAct(SAT_bins)),
    ('feature_residency', FeatureStateResident()),
    ('feature_parent_education', FeatureParentEducation()),
    ('feature_academic_load', FeatureAcadLoad()),
    ('feature_family_income', FeatureFamilyIncome()),
    ('courses_warning_signals', FeatureCourseWarningSignals(term_class[['STDNT_ID', 'TERM_CD', 'CRSE_GRD_INPUT_CD']])),
   
])

# Fit and transform your data using the pipeline
features_df = features_pipeline.fit_transform(sample_df)

In [19]:
features_df.shape

(47616, 134)

In [20]:
features_df.to_csv('features_df.csv', index=False)